### Entropy


Intuitively, we can think of **entropy** as the measure of disorder in a system. 

This set, $S$, for example, is very disordered:

<img src="https://www.evernote.com/l/AAEZbuSslJRCwKxfZaSIBJaIZqcQ-mTe9RQB/image.png" width=400px>


Let us consider the elements of $S$ that have a black border.

In [1]:
%run items.py

In [2]:
import numpy as np
import pandas as pd

items_df = pd.DataFrame(items)
items_df

,border,color,form,letter
0,False,blue,square,E
1,True,red,circle,A
2,False,green,circle,B
3,True,red,diamond,E
4,True,red,diamond,E
5,False,blue,diamond,C
6,False,green,square,A
7,True,green,diamond,D
8,True,silver,star,C
9,False,silver,square,C


## Information Entropy


We can measure the disorder in $S$ relative to the any attribute of an element using the Shannon Information **Entropy**

$$H(S) = -p_1\log_2p_1 + \dots + -p_n\log_2p_n$$




Here, each $p_i$ is a measure of the proportion of the set represented by each class for a given attribute. If we are looking at the color of our shapes, this would be five classes: 

In [3]:
items_df.color.unique()

array(['blue', 'red', 'green', 'silver', 'purple'], dtype=object)

### Calculate the Proportions

In [4]:
total_count = items_df.color.count()

In [6]:
p_red = (items_df.color == 'red').sum()/total_count
p_blue = (items_df.color == 'blue').sum()/total_count
p_green = (items_df.color == 'green').sum()/total_count
p_silver = (items_df.color == 'silver').sum()/total_count
p_purple = (items_df.color == 'purple').sum()/total_count

S_proportions = [p_red, p_blue, p_green, p_silver, p_purple]

In [7]:
S_proportions

[0.23809523809523808,
 0.2857142857142857,
 0.2857142857142857,
 0.14285714285714285,
 0.047619047619047616]

In [8]:
sum(S_proportions)

1.0

$$H(S) = -p_1\log_2p_1 + \dots + -p_n\log_2p_n$$

### $\log_2$

    np.log2()

    Base-2 logarithm of `x`.

In [10]:
-p_red*np.log2(p_red) \
    -p_blue*np.log2(p_blue) \
    -p_green*np.log2(p_green) \
    -p_silver*np.log2(p_silver) \
    -p_purple*np.log2(p_purple)

2.1359327568142765

In [15]:
def entropy(proportions):
    H_i_list = list()
    for proportion in proportions:
        H_i_list.append(-proportion*np.log2(proportion))
    return sum(H_i_list)

In [21]:
def entropy_np(proportions):
    proportions = np.array(proportions)
    log_2_proportions = np.log2(proportions)
    return -proportions.dot(log_2_proportions)

In [22]:
entropy(S_proportions), entropy_np(S_proportions)

(2.1359327568142765, 2.1359327568142765)

We can use Python to calculate the entropy

### Measure The Entropy of the color of $S$ 

## To Do

Write a method that calculates the class proportions for a given dataframe and feature.


In [31]:
items_df['color'].value_counts()/21

green     0.285714
blue      0.285714
red       0.238095
silver    0.142857
purple    0.047619
Name: color, dtype: float64

In [32]:
def class_proportions(dataframe, feature):
    if type(feature) is not str:
        raise TypeError
        
    if feature not in dataframe.columns:
        raise AttributeError
    
    total_count = dataframe[feature].count()
    class_counts = dataframe[feature].value_counts()
    
    return class_counts/total_count

Use this method to calculate the Entropy with respect to each attribute of $S$

In [33]:
color_proportions = class_proportions(items_df, 'color')
form_proporations = class_proportions(items_df, 'form')
letter_proportions = class_proportions(items_df, 'letter')
border_proportions = class_proportions(items_df, 'border')

In [36]:
from IPython.display import display

display(entropy(color_proportions), entropy(S_proportions))
display(entropy(form_proporations))
display(entropy(letter_proportions))
display(entropy(border_proportions))

2.1359327568142765

2.1359327568142765

1.938708497286834

2.2576068788115968

0.99836367259381298

In [37]:
entropy(class_proportions(items_df, 'border'))

0.99836367259381298

## The Decision Tree

Decision Trees are supervised learning models typically split into classification trees and regression trees. For the rest of this lesson, we will focus on classification trees. 

We will work through the ID3 algorithm for learning a decision tree from a set of labeled data. 

#### Labeled Data

For our purposes, let us assume that the feature `border` is our label. We will be seeking a decision tree that makes splits in order to develop a model for identifying which items will have a border. 

In [38]:
features = items_df.drop('border', axis=1)
target = items_df['border']

### Adapt `class_proportions` to take a `target`

In [39]:
def class_proportions(target):
    
    total_count = target.count()
    class_counts = target.value_counts()
    
    return class_counts/total_count

In [42]:
class_proportions(target)
entropy(class_proportions(target))

0.99836367259381298

In [41]:
display(class_proportions(target))
display(entropy(class_proportions(target)))

True     0.52381
False    0.47619
Name: border, dtype: float64

0.99836367259381298

### Three Partitioning Schemes

We can start by separating the elements based upon their attributes. Here are three different ways to do that:


![](https://www.evernote.com/l/AAFjiCSRo6xAeYC-RZ_9sy8ek0Aa69CXBDgB/image.png)

### Identify the Best Split

In order to proceed, we will need to identify which of these ways of separating is best? We can use the measure of entropy to do this!


#### Split by Shape

In [43]:
features_square_df = features[features.form == 'square']
features_circle_df = features[features.form == 'circle']
features_diamond_df = features[features.form == 'diamond']
features_star_df = features[features.form == 'star']
target_square_df = target[features.form == 'square']
target_circle_df = target[features.form == 'circle']
target_diamond_df = target[features.form == 'diamond']
target_star_df = target[features.form == 'star']

display(features_square_df)
display(target_square_df)
display(features_circle_df)
display(target_circle_df)
display(features_diamond_df)
display(target_diamond_df)
display(features_star_df)
display(target_star_df)

,color,form,letter
0,blue,square,E
6,green,square,A
9,silver,square,C
12,green,square,A
16,blue,square,E
19,purple,square,C
20,silver,square,C


0     False
6     False
9     False
12    False
16    False
19    False
20    False
Name: border, dtype: bool

,color,form,letter
1,red,circle,A
2,green,circle,B
10,green,circle,B
11,red,circle,A
13,green,circle,D
15,blue,circle,B


1      True
2     False
10    False
11     True
13     True
15     True
Name: border, dtype: bool

,color,form,letter
3,red,diamond,E
4,red,diamond,E
5,blue,diamond,C
7,green,diamond,D
17,red,diamond,E


3      True
4      True
5     False
7      True
17     True
Name: border, dtype: bool

,color,form,letter
8,silver,star,C
14,blue,star,B
18,blue,star,B


8     True
14    True
18    True
Name: border, dtype: bool

To assess this split, we will be seeking the entropy associated with each subset as a proportion of the total set.

$$H_T = q_{\text{square}}H(S_{\text{square}}) + 
        q_{\text{circle}}H(S_{\text{circle}}) + 
        q_{\text{diamond}}H(S_{\text{diamond}}) + 
        q_{\text{star}}H(S_{\text{star}})
$$

Here each $q_i$ represents the **weight**, the proportion of the total each subset is

$$q_i = \frac{\text{count}(S_i)}{\text{count}(S_T)}$$

In [44]:
q_square = features_square_df.form.count()/features.form.count()
q_circle = features_circle_df.form.count()/features.form.count()
q_diamond = features_diamond_df.form.count()/features.form.count()
q_star = features_star_df.form.count()/features.form.count()

In [56]:
q_square,q_circle,q_diamond,q_star, sum([q_square,q_circle,q_diamond,q_star])

(0.33333333333333331,
 0.2857142857142857,
 0.23809523809523808,
 0.14285714285714285,
 1.0)

### Total Entropy When Split by Form

##### Calculate the weights of each class of our target

In [45]:
square_prop = class_proportions(target_square_df)
circle_prop = class_proportions(target_circle_df)
diamond_prop = class_proportions(target_diamond_df)
star_prop = class_proportions(target_star_df)

In [65]:
square_prop, star_prop

(False    1.0
 Name: border, dtype: float64, True    1.0
 Name: border, dtype: float64)

##### Find the entropy for these proportions

In [51]:
H_square_border = entropy(square_prop)
H_circle_border = entropy(circle_prop)
H_diamond_border = entropy(diamond_prop)
H_star_border = entropy(star_prop)

In [53]:
H_square_border,H_circle_border,H_diamond_border,H_star_border

(0.0, 0.91829583405448956, 0.72192809488736231, 0.0)

In [ ]:
q_s

##### Sum the weighted Entropy

In [57]:
q_square*H_square_border + \
    q_circle*H_circle_border + \
    q_diamond*H_diamond_border + \
    q_star*H_star_border

0.43425787994113085

### Write a function to do this for a split on any feature

In [68]:
def entropy_on_split(dataframe, target, feature, debug=False):
    
    # split on feature
    unique_classes = dataframe[feature].unique()
    target_subsets = [
        target[dataframe[feature] == unique_class]
        for unique_class in unique_classes
    ]
    
    if debug: display(target_subsets)
    
    # calculate subset weights
    
    total = target.count()
    weights = [
        target_subset.count()/total 
        for target_subset in target_subsets
    ]
    
    if debug: display(weights)
    
    # calculate target proprotions
    proportions = [
        class_proportions(target_subset)
        for target_subset in target_subsets
    ]
    
    if debug: display(proportions)
    
    # calculate the entropies
    entropies = [
        entropy(proportion)
        for proportion in proportions
    ]

    if debug: display(entropies)
        
    weighted_entropies = [
        weight*entropy 
        for weight, entropy in zip(weights, entropies)
    ]

    if debug: display(weighted_entropies)    

    return sum(weighted_entropies)

Manually calculated entropy on 'form' split:

    0.43425787994113085

In [70]:
entropy_on_split(features, target, 'form')

0.43425787994113085

In [71]:
entropy_on_split(features, target, 'color')

0.67926964316620975

In [72]:
entropy_on_split(features, target, 'letter')

0.82472125804683316

### Write a function to Identify the Best Split

In [75]:
def find_best_split(features, target):
    feature_labels = features.columns
    
    entropies = [
        entropy_on_split(features, target, feature_label)
        for feature_label in feature_labels
    ]
    
    best_index = np.argmin(entropies)
    return feature_labels[best_index]

In [76]:
find_best_split(features, target)

'form'

## Representing the Tree

The ability to find a best split will only work at a single node. In order to build a complete machine learning model, we are going to need to use this method to build an entire decision tree.

We have this so far:

<img src="https://www.evernote.com/l/AAFzXwQ0IPVAcYpRMWQuaQCf0NvtOC4mz0sB/image.png" width=400px>

But what about the rest of the tree? We want to split like this:

<img src="https://www.evernote.com/l/AAHMTpccFvtOjYGojh7sZ97oDC6-bRNAPe4B/image.png" width=400px>

<img src="https://www.evernote.com/l/AAGE0WSC6F5AzLCUX-A3cwXrQULf_uFRyFYB/image.png" width=400px>

In order to represent this using Python, we will define a `tree` to be one of these:

- `True`
- `False`
- a tuple `(attribute, subtree_dict)`

For example, consider the tree representing green circles containing the letter 'B'.

We can represent this trivially as:

In [77]:
green_circle_B_tree = False

or the tree representing green circles containing the letter 'D':

In [78]:
green_circle_D_tree = True

From there, consider the tree representing green circles:

<img src="https://www.evernote.com/l/AAEbErEwS7hETI2gNXzkaetSXntpxeGKjSoB/image.png" width=200px>

In [ ]:
green_circle_tree = ('letter', {'B' : False, 'D' : True})

We might gradually work our way up from there and represent our entire tree as

In [79]:
tree = ('form', {'square': False,
                 'circle': 
                           ('color', {'green': 
                                               ('letter', {'B' : False, 
                                                           'D' : True}),
                                      'red': True,
                                      'blue': True}),
                 'diamond': 
                            ('letter', {'C': False,
                                        'D': True,
                                        'E': True}),
                 'star': True})

### Use a tree to classifiy an input

Suppose we had a new element and we wish to no whether or not is has a border. For example, we may be given the following element:

    {'form': 'circle', 'letter': 'C', 'color': 'red'}

We are going to need to build a classification function to use our tree to classify this input.

### Recursion

A recursive function is a function that calls itself.

$$F_0 = 0$$
$$F_1 = 1$$
$$F_2 = 1$$
$$F_3 = 2$$
$$F_4 = 3$$
$$F_5 = 5$$
$$F_6 = 8$$
$$F_7 = 13$$

In [83]:
def fib(n):
    if n in [0,1]:
        return n
        
    return fib(n-2) + fib(n-1)

In [84]:
fib(6), fib(7)

(8, 13)

### Recursively Traverse the Tree to find the class

In [85]:
def classify(tree, element):
    
    # recursion end case
    if tree in [True, False]:
        return tree
    
    # if tuple defining a subtree dictionary
    # attribute defines the split
    # subtree dictionary defines the subtrees created by that split
    attribute, subtree_dictionary = tree
    
    # get the attribute from the element associated with that split
    subtree_key = element.get(attribute)
    
    # get the subtree associated with key
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)

If we don't get `True` or `False` we will get a tuple representing a subtree dictionary:

    ('letter', {'B' : False, 'D' : True})

In [86]:
this_element = {'form': 'circle', 'letter': 'C', 'color': 'red'}
classify(tree, this_element)

True

But what if we pass an element that is ambiguous, for example

    {'form': 'circle', 'letter': 'C', 'color': 'green'}

In [87]:
other_element = {'form': 'circle', 'letter': 'C', 'color': 'green'}
classify(tree, other_element)

KeyError: 'C'

### To Handle this, we will redefine our Tree

We will add a `None` key that returns the most common class.

In [88]:
tree = ('form', {'square': False,
                 'circle': 
                           ('color', {'green': 
                                               ('letter', {'B' : False, 
                                                           'D' : True,
                                                           None: False}),
                                      'red': True,
                                      'blue': True,
                                      None : True}),
                 'diamond': 
                            ('letter', {'C': False,
                                        'D': True,
                                        'E': True,
                                        None: True}),
                 'star': True,
                 None: True})

In [89]:
def classify(tree, element):
    
    # recursion end case
    if tree in [True, False]:
        return tree
    
    # if tuple defining a subtree dictionary
    # attribute defines the split
    # subtree dictionary defines the subtrees created by that split
    attribute, subtree_dictionary = tree
    
    # get the attribute from the element associated with that split
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    # get the subtree associated with key
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)

In [90]:
classify(tree, {'form': 'circle', 'letter': 'C', 'color': 'green'})

False

In [91]:
classify(tree, {'form': 'octagon', 'letter': 'Z', 'color': 'chartreuse'})

True

## Build The Tree

In [92]:
def find_best_split(features, target, split_candidates):
    entropies = [
        entropy_on_split(features, target, split_candidate)
        for split_candidate in split_candidates
    ]
    
    best_index = np.argmin(entropies)
    return split_candidates[best_index]

In [98]:
def build_tree(features, target, split_candidates=None):
    # handle the case for split_candidates is None
    if split_candidates is None:
        split_candidates = list(features.columns)
        
    total_count = target.count()
    true_count = target.sum()
    false_count = total_count - true_count
    
    if false_count == 0: return True
    if true_count == 0: return False
    
    if split_candidates == []:
        return true_count > false_count
    
    # find the best attribute to split upon
    best_attribute = find_best_split(features, target, split_candidates)
    
    # remove best attribute from list of split candidates
    split_candidates = [split_candidate
                        for split_candidate in split_candidates
                        if split_candidate is not best_attribute]
    
    best_attribute_uniques = features[best_attribute].unique()
    
    subtree_dictionary = dict()
    
    for best_attribute_unique in best_attribute_uniques:
        feat_subset_mask = features[best_attribute] == best_attribute_unique
        feat_subset = features[feat_subset_mask]
        target_subset = target[feat_subset_mask]
        
        subtree_dictionary[best_attribute_unique] = \
            build_tree(feat_subset, target_subset, split_candidates)

    subtree_dictionary[None] = true_count > false_count
            
    return (best_attribute, subtree_dictionary)

In [106]:
build_tree(features, target)

('form',
 {None: True,
  'circle': ('color',
   {None: True,
    'blue': True,
    'green': ('letter', {'B': False, 'D': True, None: False}),
    'red': True}),
  'diamond': ('color',
   {None: True, 'blue': False, 'green': True, 'red': True}),
  'square': False,
  'star': True})

In [102]:
tree

('form',
 {None: True,
  'circle': ('color',
   {None: True,
    'blue': True,
    'green': ('letter', {'B': False, 'D': True, None: False}),
    'red': True}),
  'diamond': ('letter', {'C': False, 'D': True, 'E': True, None: True}),
  'square': False,
  'star': True})

In [107]:
my_tree = build_tree(features, target)

In [108]:
display(classify(my_tree, {'form': 'circle', 'letter': 'C', 'color': 'red'}))
display(classify(my_tree, {'form': 'circle', 'letter': 'C', 'color': 'green'}))
display(classify(my_tree, {'form': 'octagon', 'letter': 'Z', 'color': 'chartreuse'}))

True

False

True